$\Delta u + k^2(1+q) u = f $ in $\Omega = [0,1]^2$    
$u = 0 $ on $\partial \Omega$

In [1]:
import scipy as sp
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from scipy.sparse import csc_matrix
from scipy.sparse.linalg import gmres,spilu,LinearOperator,lgmres
import time
import random

In [2]:
k = 1  # wavenumber
N = 100  # 格点数
h = 1 / N  # 间隔

In [3]:
def q_gen_example(N):
    q = np.zeros((N + 1, N + 1))
    q_value = 0.02
    x1, x2, x3, y1, y2, y3, y4 = 0.2, 0.4, 0.7, 0.2, 0.3, 0.6, 0.7
    q[int(x1 * N):int(x2 * N), int(y1 * N):int(y4 * N)] = q_value
    q[int(x2 * N):int(x3 * N), int(y2 * N):int(y3 * N)] = q_value
    return q

def q_gen_1(N,b1 = 0.3,b2 = 0.6,a1 = 8,a2 = 9,gamma = 1): # 𝑞(𝑥,𝑦)= 𝜆   exp⁡(−𝑎1(𝑥−𝑏1 )^2−a2 (y−b2 )^2 ) 

    q = np.zeros((N + 1, N + 1))
    for i in range(1,N):
        for j in range(1,N):
            q[i,j] = gamma* np.exp(-a1*(i/N - b1)**2-a2*(j/N - b2)**2)
    return q

def q_generation(N, method='T'): 
    if method == 'T':
        return q_gen_example(N)
    elif method == 'Gauss':
        return q_gen_1(N)
    print('method error')


q = q_generation(N)
# sns.heatmap(q, xticklabels=False, yticklabels=False)

$u = \sin(x\pi)\sin(y\pi)$  
$f = \Delta u + (1+q) u = (1+q-2\pi^2)u$

In [4]:
def u_gen(N):
    u = np.zeros((N+1,N+1))
    for i in range(1,N):
        for j in range(1,N):
            u[i,j] = np.sin(i*np.pi/N)*np.sin(j*np.pi/N)
    return u

u_truth = u_gen(N)
# sns.heatmap(u_truth, xticklabels=False, yticklabels=False)

In [5]:
def f_gen_1(N,q,u,k = 1):
    return ((1+q)*k*k-2*np.pi*np.pi)*u

f = f_gen_1(N,q,u_truth)
# sns.heatmap(f, xticklabels=False, yticklabels=False)

###  九点格式
$A \,u_{i,j} = u_{i+1,j} +u_{i-1,j} +u_{i,j+1} +u_{i,j-1}  $  
$\frac{A-4I}{h^2} u_{i,j} + k^2(1+q_{i,j})u_{i,j} = f_{i,j} $ 
$B \,u_{i,j} = u_{i+1,j+1} +u_{i-1,j-1} +u_{i-1,j+1} +u_{i+1,j-1}$  
$\frac{A-4I}{h^2} u_{i,j} + \frac{B - 2A+4I}{6h^2}u_{i,j} + k^2(1+q_{i,j})u_{i,j} + \frac{k^2}{12}(A-4I)
(1+q_{i,j})u_{i,j}= f_{i,j} + \frac{1}{12}(A-4I)f_{i,j}$  
$\Rightarrow\quad (B+4A-20I) u_{i,j} +h^2 k^2(0.5A+4I)
(1+q_{i,j})u_{i,j}= h^2 (0.5A+4I)f_{i,j}$

In [6]:
def A(v):
    v[0, :] = 0
    v[-1, :] = 0
    v[:, 0] = 0
    v[:, -1] = 0
    return v[1:-1, 2:] + v[:-2, 1:-1] + v[2:, 1:-1] + v[1:-1, :-2]


def Matrix_9(N, q, k=1):
    M = N - 1
    h = 1 / N

    Q = q[1:-1, 1:-1].reshape(1, -1)

    value_1 = (1 + Q) * h * h * k * k * 4 - 20  # 主对角线
    value_2 = 4 + 0.5 * h * h * k * k * (1 + q[1:-1, 1:-1])  # A 三对角线&主对角元三对角线

    data_main = value_1  # 主对角线
    data_main_up = np.reshape(np.c_[np.zeros((M, 1)), value_2[:, :-1]],
                              (1, -1))  # 主对角元上对角线
    data_main_down = np.reshape(np.c_[value_2[:, 1:],
                                      np.zeros((M, 1))], (1, -1))  # 主对角元下对角线
    data_up = np.c_[np.zeros((1, M)), value_2[:-1, :].reshape(1,
                                                              -1)]  # 上对角元对角线
    data_up_up = np.tile(np.insert(np.ones(M - 1), 0, 0),
                         M).reshape(1, -1)  # 上对角元上对角线
    data_up_down = np.tile(np.append(np.ones(M - 1), 0),
                           M).reshape(1, -1)  # 上对角元下对角线
    data_down = np.c_[value_2[1:, :].reshape(1, -1), np.zeros((1, M))]  # 下对角元
    data_down_up = data_up_up  # 下对角元上对角线
    data_down_down = data_up_down  # 下对角元下对角线
    data = np.r_[data_main, data_main_up, data_main_down, data_up, data_up_up,
                 data_up_down, data_down, data_down_up, data_down_down]
    offsets = np.array([0, 1, -1, M, M + 1, M - 1, -M, -M + 1, -M - 1])
    dia = sp.sparse.dia_matrix((data, offsets), shape=(M * M, M * M))
    return dia


In [7]:
def Generate(N,k=1,method = 'Gauss',q_value = None,u_truth = u_truth):
    h = 1/N
    if q_value.all()==None:
        q = q_generation(N,method)
    else:
        q = q_value
    
    f = f_gen_1(N, q, u_truth,k = k)
    Matrix9 = Matrix_9(N, q,k = k)
    Right9 = ((0.5 * A(f) + 4 * f[1:-1, 1:-1]) * h * h).reshape((-1, 1))
    return Matrix9,Right9

def Error(a, a_truth, gap=1e-10):
    a1 = np.where(a < gap, gap, a)
    a_t1 = np.where(a_truth < gap, gap, a_truth)
    return np.abs(a1 / a_t1 - 1)

In [8]:
def perform(N, u_truth = u_truth,q_method = 'T',k = 1,tol=1e-05, inner_m=30,q_value = None):


    Matrix9,Right9 = Generate(N,k,q_method,q_value,u_truth)
    
    # GMRES
    time90 = time.time()
    u_res, exit = gmres(Matrix9, Right9, tol=tol,restart=inner_m)
    time91 = time.time() - time90
    if exit == 0:
        res9 = np.zeros((N + 1, N + 1))
        res9[1:-1, 1:-1] = u_res.reshape(N - 1, N - 1)
        err90 = np.linalg.norm(Error(res9, u_truth), ord=2) / (N - 1)
    else:
        print('GMRES不收敛')
    
    
    return err90,time91


In [9]:
def samp(m,N): #m 样本量
    ll = []
    for i in range(m):
        n = random.randint(1,5)
        q_value = np.zeros((N+1,N+1))
        for j in range(n):
            a1 = random.uniform(5,10)
            a2 = random.uniform(5,10)
            b1 = random.uniform(0.2,0.8)
            b2 = random.uniform(0.2,0.8)
            gamma = random.uniform(-0.5,0.5)
            q_value += q_gen_1(N,b1,b2,a1,a2,gamma)
        ll.append(q_value)
    return ll

In [10]:
def total_gen(NN,tol,k=2,samples = 100,inner_m = 100):
    u_truth = u_gen(NN)
    res = samp(samples,NN)
    err,time0 = 0,0
    for i in range(samples):
        tmp = perform(NN, u_truth=u_truth,q_value = res[i],k = 2,tol=tol, inner_m=inner_m)
        err+=tmp[0]
        time0+=tmp[1]
    print('samples=%d,N=%d,tol=%f' %(samples,NN,tol))    
    print(err/samples,time0/samples)
    

In [11]:
total_gen(50,1e-02)

samples=100,N=50,tol=0.010000
0.0030484971598082667 0.0011551976203918457


In [12]:
total_gen(50,1e-04)

samples=100,N=50,tol=0.000100
2.8337303548466236e-05 0.0029746341705322267


In [13]:
total_gen(100,1e-02)

samples=100,N=100,tol=0.010000
0.0032402911665719092 0.008999567031860351


In [14]:
total_gen(100,1e-04)

samples=100,N=100,tol=0.000100
2.830737128221588e-05 0.031043882369995116


In [15]:
total_gen(200,1e-02)

samples=100,N=200,tol=0.010000
0.003490637151102476 0.08999395608901978


In [16]:
total_gen(200,1e-04)

samples=100,N=200,tol=0.000100
5.763111315676516e-05 0.27897504329681394
